## Imports

In [1]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from string import punctuation
import json
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text  import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.tree import DecisionTreeClassifier
import pickle
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
pd.set_option('display.max_columns', None)

## Loading and exploring dataset <br>
[Dataset link]('https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset?select=movies_metadata.csv')

In [3]:
df = pd.read_csv('movies_metadata.csv', low_memory=False)
df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [4]:
df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [5]:
print(df['genres'][1])
print(type(df['genres'][1]))

[{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 10751, 'name': 'Family'}]
<class 'str'>


In [6]:
df[['original_title' , 'genres', 'tagline', 'overview']].isna().sum()

original_title        0
genres                0
tagline           25054
overview            954
dtype: int64

## Data cleaning

After I explored the dataset. I found many NaN and unwanted column - for our pupose,  in the dataset. So, I made subset of that are required. and will remove the Rows that  contain NaN<br>
`genres`, `tagline`, `overview`

In [7]:
df_subset = df[['genres', 'tagline', 'overview']]
df_subset.head()

,genres,tagline,overview
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",NaN,"Led by Woody, Andy's toys live happily in his ..."
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",Roll the dice and unleash the excitement!,When siblings Judy and Peter discover an encha...
2,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",Still Yelling. Still Fighting. Still Ready for...,A family wedding reignites the ancient feud be...
3,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",Friends are the people who let you be yourself...,"Cheated on, mistreated and stepped on, the wom..."
4,"[{'id': 35, 'name': 'Comedy'}]",Just When His World Is Back To Normal... He's ...,Just when George Banks has recovered from his ...


In [8]:
df_subset = df_subset.reset_index(drop=True)
print(df_subset.shape)
df_subset.head()

(45466, 3)


,genres,tagline,overview
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",NaN,"Led by Woody, Andy's toys live happily in his ..."
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",Roll the dice and unleash the excitement!,When siblings Judy and Peter discover an encha...
2,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",Still Yelling. Still Fighting. Still Ready for...,A family wedding reignites the ancient feud be...
3,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",Friends are the people who let you be yourself...,"Cheated on, mistreated and stepped on, the wom..."
4,"[{'id': 35, 'name': 'Comedy'}]",Just When His World Is Back To Normal... He's ...,Just when George Banks has recovered from his ...


Droping rows containing NaN values

In [9]:
df_subset = df_subset.dropna()
df_subset.shape
df_subset

,genres,tagline,overview
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",Roll the dice and unleash the excitement!,When siblings Judy and Peter discover an encha...
2,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",Still Yelling. Still Fighting. Still Ready for...,A family wedding reignites the ancient feud be...
3,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",Friends are the people who let you be yourself...,"Cheated on, mistreated and stepped on, the wom..."
4,"[{'id': 35, 'name': 'Comedy'}]",Just When His World Is Back To Normal... He's ...,Just when George Banks has recovered from his ...
5,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",A Los Angeles Crime Saga,"Obsessive master thief, Neil McCauley leads a ..."
...,...,...,...
45454,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",What happened last night?,The Morning After is a feature film that consi...
45456,"[{'id': 27, 'name': 'Horror'}, {'id': 9648, 'n...",Meet...The CREEPER!,An unsuccessful sculptor saves a madman named ...
45458,"[{'id': 27, 'name': 'Horror'}]","Do you know what happened 50 years before ""The...",A film archivist revisits the story of Rustin ...
45461,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",Rising and falling between a man and woman,Rising and falling between a man and woman.


Converting the genre column to string

In [10]:
def handel_genre(text):
    text = text.replace("'", '"')
    data = json.loads(text)
    try:
        data = data[0]['name'].lower()
    except Exception as e:
        data = np.nan
    return data


.apply() function in pandas is used to apply a function in dataframe collumn  <br>
I have updated the 'genres' column with new one.

* "[{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 10751, 'name': 'Family'}]" <br>
TO <br>
* Adventure Fantasy Family

In [11]:
df_subset['genres'] = df_subset['genres'].apply(handel_genre)
df_subset.head()

,genres,tagline,overview
1,adventure,Roll the dice and unleash the excitement!,When siblings Judy and Peter discover an encha...
2,romance,Still Yelling. Still Fighting. Still Ready for...,A family wedding reignites the ancient feud be...
3,comedy,Friends are the people who let you be yourself...,"Cheated on, mistreated and stepped on, the wom..."
4,comedy,Just When His World Is Back To Normal... He's ...,Just when George Banks has recovered from his ...
5,action,A Los Angeles Crime Saga,"Obsessive master thief, Neil McCauley leads a ..."


## Making the function to tokenize the texts and implementing in dataset.

- We have to tokenize the text while doing NLP because every words do not give us the context of sentence. <Br>
for instance, `he` `is` `a` boy`.` ONLY boy is our tag mean which gives us the general context. Same goes with punctuation. <br>
This will help in reasulting better accuray in most cases and reduce computations.

In [12]:
def tokenize(text):
    stop_words = set(stopwords.words('english'))
    text_words = word_tokenize(text, 'english')
    text_token = [tokens for tokens in text_words if tokens.lower() not in stop_words and  tokens.lower() not in punctuation]
    return (' '.join(text_token)).lower()

In [13]:
df_subset['tagline'] = df_subset['tagline'].apply(tokenize)
df_subset['overview'] = df_subset['overview'].apply(tokenize)
df_subset.head()

,genres,tagline,overview
1,adventure,roll dice unleash excitement,siblings judy peter discover enchanted board g...
2,romance,still yelling still fighting still ready love,family wedding reignites ancient feud next-doo...
3,comedy,friends people let ... never let forget,cheated mistreated stepped women holding breat...
4,comedy,world back normal ... 's surprise life,george banks recovered daughter 's wedding rec...
5,action,los angeles crime saga,obsessive master thief neil mccauley leads top...


In [14]:
df_subset = df_subset.dropna()
df_subset.shape

(20139, 3)

## Encoding genres For classification

In [15]:
# mlb = MultiLabelBinarizer()
# genre_encoded = pd.DataFrame(mlb.fit_transform(df_subset['genres'].apply(lambda x: x.split())), columns=mlb.classes_)
# df_subset = pd.concat([df_subset, genre_encoded], axis=1)
# df_subset = df_subset.drop('genres', axis=1)
# df_subset.head()

## Vectorizer

As text of both overview and tagline can be used for prediction I merged it together for simple approch

In [16]:
df_subset['combined_text'] = df_subset['tagline'] + ' ' + df_subset['overview']
df_subset.head()

C:\Users\ACER\AppData\Local\Temp\ipykernel_4464\1359123265.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['combined_text'] = df_subset['tagline'] + ' ' + df_subset['overview']


,genres,tagline,overview,combined_text
1,adventure,roll dice unleash excitement,siblings judy peter discover enchanted board g...,roll dice unleash excitement siblings judy pet...
2,romance,still yelling still fighting still ready love,family wedding reignites ancient feud next-doo...,still yelling still fighting still ready love ...
3,comedy,friends people let ... never let forget,cheated mistreated stepped women holding breat...,friends people let ... never let forget cheate...
4,comedy,world back normal ... 's surprise life,george banks recovered daughter 's wedding rec...,world back normal ... 's surprise life george ...
5,action,los angeles crime saga,obsessive master thief neil mccauley leads top...,los angeles crime saga obsessive master thief ...


In [17]:
df_subset.head(1)

,genres,tagline,overview,combined_text
1,adventure,roll dice unleash excitement,siblings judy peter discover enchanted board g...,roll dice unleash excitement siblings judy pet...


Vectorizing using TFID

In [18]:
vect = TfidfVectorizer(max_features=1500,  ngram_range=(1, 4))
vect_features = vect.fit_transform(df_subset['combined_text'])

In [37]:
vect.vocabulary_

{'roll': 1109,
 'peter': 981,
 'discover': 358,
 'board': 147,
 'game': 531,
 'door': 368,
 'magical': 807,
 'world': 1474,
 'adult': 21,
 'trapped': 1345,
 'inside': 667,
 'years': 1487,
 'living': 771,
 'room': 1112,
 'hope': 625,
 'freedom': 521,
 'proves': 1034,
 'three': 1322,
 'find': 489,
 'running': 1117,
 'giant': 547,
 'evil': 429,
 'terrifying': 1303,
 'creatures': 294,
 'still': 1236,
 'fighting': 478,
 'ready': 1055,
 'love': 793,
 'family': 455,
 'wedding': 1438,
 'ancient': 56,
 'next': 916,
 'john': 698,
 'max': 830,
 'meanwhile': 835,
 'italian': 680,
 'local': 773,
 'shop': 1181,
 'll': 772,
 'away': 98,
 'less': 757,
 'hot': 632,
 'time': 1325,
 'friends': 524,
 'people': 977,
 'let': 758,
 'never': 909,
 'forget': 511,
 'women': 1465,
 'waiting': 1422,
 'good': 563,
 'man': 813,
 'break': 161,
 'lovers': 797,
 'talk': 1286,
 'determined': 346,
 'better': 132,
 'way': 1433,
 'back': 100,
 'normal': 921,
 'life': 761,
 'george': 540,
 'daughter': 317,
 'receives': 106

In [19]:
vect.get_feature_names_out()

array(['000', '10', '11', ..., 'young woman', 'younger', 'youth'],
      dtype=object)

In [20]:
tfidf_df = pd.DataFrame(vect_features.toarray(), columns=vect.get_feature_names_out())
df_subset = df_subset.drop(['tagline', 'overview', 'combined_text'], axis=1)

In [21]:
tfidf_df.head()

,000,10,11,12,15,20,30,abandoned,able,academy,accident,accidentally,accused,across,act,action,actor,actress,actually,adam,adaptation,adult,adventure,adventures,affair,africa,african,age,aged,agent,agents,aging,ago,agrees,aid,air,alex,alice,alien,aliens,alive,all,almost,alone,along,along way,already,also,although,always,amazing,ambitious,america,american,among,amp,ancient,and,angel,angeles,animal,animals,animated,anna,another,anyone,anything,apart,apartment,appears,area,armed,arms,army,around,arrival,arrive,arrives,art,arthur,artist,arts,asks,aspiring,assassin,assigned,assistant,attack,attacks,attempt,attempts,attention,attorney,australian,author,authorities,avoid,award,away,baby,back,bad,band,bank,bar,based,based true,based true story,battle,battles,beach,beast,beautiful,beauty,became,become,becomes,becoming,befriends,began,begin,beginning,begins,behind,believe,believes,beloved,ben,best,best friend,best friends,betrayal,better,beyond,big,biggest,bill,billy,birth,birthday,bit,bizarre,black,blind,blood,bloody,blue,board,boat,bob,bodies,body,bomb,bond,book,born,boss,boy,boyfriend,boys,brain,break,breaks,brian,bride,brilliant,bring,brings,british,broadway,broken,brother,brothers,brought,brutal,buddy,build,building,buried,business,businessman,buy,ca,california,call,called,calls,came,camp,captain,capture,captured,car,care,career,case,cash,cast,castle,cat,catch,caught,cause,causes,center,century,chain,challenge,champion,chance,change,changed,changes,chaos,character,characters,charge,charles,charlie,charming,chase,chicago,chief,child,childhood,children,china,chinese,choice,choose,chris,christmas,church,cia,city,civil,civil war,claim,claims,class,classic,clear,close,closer,club,co,coach,coast,cold,college,come,comedy,comes,comic,coming,commit,committed,common,community,company,competition,complete,complicated,computer,con,concert,conflict,confront,connection,consequences,conspiracy,contact,continues,control,convince,convinced,cop,cops,corporate,corrupt,could,count,country,couple,courage,course,court,cousin,cover,crash,crazy,create,created,creature,creatures,crew,crime,crimes,criminal,criminals,crisis,cross,cult,culture,curse,cut,dad,dance,dancer,danger,dangerous,daniel,danny,daring,dark,darkness,date,daughter,daughters,david,day,days,de,dead,deadly,deal,dealer,dealing,death,deaths,decide,decides,deep,defeat,demon,demons,desert,desire,desperate,desperately,despite,destiny,destroy,destroyed,destruction,detective,determined,develops,devil,diamond,die,died,dies,different,difficult,directed,director,disaster,discover,discovered,discovers,discovery,disease,divorce,doctor,documentary,dog,done,door,double,dr,drama,drawn,dream,dreams,drive,driven,driver,drug,drugs,due,dying,early,earth,east,easy,eccentric,eddie,edge,effort,eight,either,elite,elizabeth,else,embark,embarks,empire,encounter,encounters,end,ends,enemies,enemy,england,english,enough,enter,enters,entire,epic,era,escape,escaped,escapes,especially,estate,estranged,europe,eve,even,event,events,eventually,ever,every,everyone,everything,evidence,evil,ex,except,executive,existence,expedition,experience,experiences,experiment,expert,explores,extraordinary,eye,eyes,face,faces,fact,failed,faith,fall,fall love,falling,falls,falls love,fame,families,family,famous,fantasy,far,farm,fashion,fast,fate,father,fbi,fear,fears,feature,features,featuring,feel,feelings,feels,fellow,female,field,fight,fighter,fighting,fights,figure,filled,film,filmed,filmmaker,films,final,finally,financial,find,find way,finding,finds,fire,first,first time,five,flesh,flight,fly,follow,following,follows,food,footage,football,force,forced,forces,forest,forever,forget,form,former,fortune,found,four,france,francisco,frank,free,freedom,french,friend,friends,friendship,front,full,fun,funny,future,game,games,gang,gangster,gave,gay,general,generation,genius,george,german,germany,get,gets,getting,ghost,giant,gift,girl,girlfriend,girls,give,given,gives,giving,global,go,god,goes,going,gold,gone,good,got,government,great,greatest,green

In [22]:
df_subset = pd.concat([df_subset.reset_index(drop=True), tfidf_df.reset_index(drop=True)], axis=1)
df_subset.head()

,genres,000,10,11,12,15,20,30,abandoned,able,academy,accident,accidentally,accused,across,act,action,actor,actress,actually,adam,adaptation,adult,adventure,adventures,affair,africa,african,age,aged,agent,agents,aging,ago,agrees,aid,air,alex,alice,alien,aliens,alive,all,almost,alone,along,along way,already,also,although,always,amazing,ambitious,america,american,among,amp,ancient,and,angel,angeles,animal,animals,animated,anna,another,anyone,anything,apart,apartment,appears,area,armed,arms,army,around,arrival,arrive,arrives,art,arthur,artist,arts,asks,aspiring,assassin,assigned,assistant,attack,attacks,attempt,attempts,attention,attorney,australian,author,authorities,avoid,award,away,baby,back,bad,band,bank,bar,based,based true,based true story,battle,battles,beach,beast,beautiful,beauty,became,become,becomes,becoming,befriends,began,begin,beginning,begins,behind,believe,believes,beloved,ben,best,best friend,best friends,betrayal,better,beyond,big,biggest,bill,billy,birth,birthday,bit,bizarre,black,blind,blood,bloody,blue,board,boat,bob,bodies,body,bomb,bond,book,born,boss,boy,boyfriend,boys,brain,break,breaks,brian,bride,brilliant,bring,brings,british,broadway,broken,brother,brothers,brought,brutal,buddy,build,building,buried,business,businessman,buy,ca,california,call,called,calls,came,camp,captain,capture,captured,car,care,career,case,cash,cast,castle,cat,catch,caught,cause,causes,center,century,chain,challenge,champion,chance,change,changed,changes,chaos,character,characters,charge,charles,charlie,charming,chase,chicago,chief,child,childhood,children,china,chinese,choice,choose,chris,christmas,church,cia,city,civil,civil war,claim,claims,class,classic,clear,close,closer,club,co,coach,coast,cold,college,come,comedy,comes,comic,coming,commit,committed,common,community,company,competition,complete,complicated,computer,con,concert,conflict,confront,connection,consequences,conspiracy,contact,continues,control,convince,convinced,cop,cops,corporate,corrupt,could,count,country,couple,courage,course,court,cousin,cover,crash,crazy,create,created,creature,creatures,crew,crime,crimes,criminal,criminals,crisis,cross,cult,culture,curse,cut,dad,dance,dancer,danger,dangerous,daniel,danny,daring,dark,darkness,date,daughter,daughters,david,day,days,de,dead,deadly,deal,dealer,dealing,death,deaths,decide,decides,deep,defeat,demon,demons,desert,desire,desperate,desperately,despite,destiny,destroy,destroyed,destruction,detective,determined,develops,devil,diamond,die,died,dies,different,difficult,directed,director,disaster,discover,discovered,discovers,discovery,disease,divorce,doctor,documentary,dog,done,door,double,dr,drama,drawn,dream,dreams,drive,driven,driver,drug,drugs,due,dying,early,earth,east,easy,eccentric,eddie,edge,effort,eight,either,elite,elizabeth,else,embark,embarks,empire,encounter,encounters,end,ends,enemies,enemy,england,english,enough,enter,enters,entire,epic,era,escape,escaped,escapes,especially,estate,estranged,europe,eve,even,event,events,eventually,ever,every,everyone,everything,evidence,evil,ex,except,executive,existence,expedition,experience,experiences,experiment,expert,explores,extraordinary,eye,eyes,face,faces,fact,failed,faith,fall,fall love,falling,falls,falls love,fame,families,family,famous,fantasy,far,farm,fashion,fast,fate,father,fbi,fear,fears,feature,features,featuring,feel,feelings,feels,fellow,female,field,fight,fighter,fighting,fights,figure,filled,film,filmed,filmmaker,films,final,finally,financial,find,find way,finding,finds,fire,first,first time,five,flesh,flight,fly,follow,following,follows,food,footage,football,force,forced,forces,forest,forever,forget,form,former,fortune,found,four,france,francisco,frank,free,freedom,french,friend,friends,friendship,front,full,fun,funny,future,game,games,gang,gangster,gave,gay,general,generation,genius,george,german,germany,get,gets,getting,ghost,giant,gift,girl,girlfriend,girls,give,given,gives,giving,global,go,god,goes,going,gold,gone,good,got,government,great,greates

In [23]:
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
ohe_tansform = ohe.fit_transform(df_subset[['genres']])
categories = ohe.categories_[0]
ohe_df = pd.DataFrame(ohe_tansform  , columns=categories)
ohe_df.head()

,action,adventure,animation,comedy,crime,documentary,drama,family,fantasy,foreign,history,horror,music,mystery,romance,science fiction,thriller,tv movie,war,western
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
df_subset = pd.concat([df_subset, ohe_df], axis=1 ).drop(columns=['genres'])

In [25]:
df_subset.head()

,000,10,11,12,15,20,30,abandoned,able,academy,accident,accidentally,accused,across,act,action,actor,actress,actually,adam,adaptation,adult,adventure,adventures,affair,africa,african,age,aged,agent,agents,aging,ago,agrees,aid,air,alex,alice,alien,aliens,alive,all,almost,alone,along,along way,already,also,although,always,amazing,ambitious,america,american,among,amp,ancient,and,angel,angeles,animal,animals,animated,anna,another,anyone,anything,apart,apartment,appears,area,armed,arms,army,around,arrival,arrive,arrives,art,arthur,artist,arts,asks,aspiring,assassin,assigned,assistant,attack,attacks,attempt,attempts,attention,attorney,australian,author,authorities,avoid,award,away,baby,back,bad,band,bank,bar,based,based true,based true story,battle,battles,beach,beast,beautiful,beauty,became,become,becomes,becoming,befriends,began,begin,beginning,begins,behind,believe,believes,beloved,ben,best,best friend,best friends,betrayal,better,beyond,big,biggest,bill,billy,birth,birthday,bit,bizarre,black,blind,blood,bloody,blue,board,boat,bob,bodies,body,bomb,bond,book,born,boss,boy,boyfriend,boys,brain,break,breaks,brian,bride,brilliant,bring,brings,british,broadway,broken,brother,brothers,brought,brutal,buddy,build,building,buried,business,businessman,buy,ca,california,call,called,calls,came,camp,captain,capture,captured,car,care,career,case,cash,cast,castle,cat,catch,caught,cause,causes,center,century,chain,challenge,champion,chance,change,changed,changes,chaos,character,characters,charge,charles,charlie,charming,chase,chicago,chief,child,childhood,children,china,chinese,choice,choose,chris,christmas,church,cia,city,civil,civil war,claim,claims,class,classic,clear,close,closer,club,co,coach,coast,cold,college,come,comedy,comes,comic,coming,commit,committed,common,community,company,competition,complete,complicated,computer,con,concert,conflict,confront,connection,consequences,conspiracy,contact,continues,control,convince,convinced,cop,cops,corporate,corrupt,could,count,country,couple,courage,course,court,cousin,cover,crash,crazy,create,created,creature,creatures,crew,crime,crimes,criminal,criminals,crisis,cross,cult,culture,curse,cut,dad,dance,dancer,danger,dangerous,daniel,danny,daring,dark,darkness,date,daughter,daughters,david,day,days,de,dead,deadly,deal,dealer,dealing,death,deaths,decide,decides,deep,defeat,demon,demons,desert,desire,desperate,desperately,despite,destiny,destroy,destroyed,destruction,detective,determined,develops,devil,diamond,die,died,dies,different,difficult,directed,director,disaster,discover,discovered,discovers,discovery,disease,divorce,doctor,documentary,dog,done,door,double,dr,drama,drawn,dream,dreams,drive,driven,driver,drug,drugs,due,dying,early,earth,east,easy,eccentric,eddie,edge,effort,eight,either,elite,elizabeth,else,embark,embarks,empire,encounter,encounters,end,ends,enemies,enemy,england,english,enough,enter,enters,entire,epic,era,escape,escaped,escapes,especially,estate,estranged,europe,eve,even,event,events,eventually,ever,every,everyone,everything,evidence,evil,ex,except,executive,existence,expedition,experience,experiences,experiment,expert,explores,extraordinary,eye,eyes,face,faces,fact,failed,faith,fall,fall love,falling,falls,falls love,fame,families,family,famous,fantasy,far,farm,fashion,fast,fate,father,fbi,fear,fears,feature,features,featuring,feel,feelings,feels,fellow,female,field,fight,fighter,fighting,fights,figure,filled,film,filmed,filmmaker,films,final,finally,financial,find,find way,finding,finds,fire,first,first time,five,flesh,flight,fly,follow,following,follows,food,footage,football,force,forced,forces,forest,forever,forget,form,former,fortune,found,four,france,francisco,frank,free,freedom,french,friend,friends,friendship,front,full,fun,funny,future,game,games,gang,gangster,gave,gay,general,generation,genius,george,german,germany,get,gets,getting,ghost,giant,gift,girl,girlfriend,girls,give,given,gives,giving,global,go,god,goes,going,gold,gone,good,got,government,great,greatest,green

In [26]:
df_subset.shape

(20139, 1520)

In [27]:
X  = df_subset.iloc[:, :1500]
X.head()

,000,10,11,12,15,20,30,abandoned,able,academy,accident,accidentally,accused,across,act,action,actor,actress,actually,adam,adaptation,adult,adventure,adventures,affair,africa,african,age,aged,agent,agents,aging,ago,agrees,aid,air,alex,alice,alien,aliens,alive,all,almost,alone,along,along way,already,also,although,always,amazing,ambitious,america,american,among,amp,ancient,and,angel,angeles,animal,animals,animated,anna,another,anyone,anything,apart,apartment,appears,area,armed,arms,army,around,arrival,arrive,arrives,art,arthur,artist,arts,asks,aspiring,assassin,assigned,assistant,attack,attacks,attempt,attempts,attention,attorney,australian,author,authorities,avoid,award,away,baby,back,bad,band,bank,bar,based,based true,based true story,battle,battles,beach,beast,beautiful,beauty,became,become,becomes,becoming,befriends,began,begin,beginning,begins,behind,believe,believes,beloved,ben,best,best friend,best friends,betrayal,better,beyond,big,biggest,bill,billy,birth,birthday,bit,bizarre,black,blind,blood,bloody,blue,board,boat,bob,bodies,body,bomb,bond,book,born,boss,boy,boyfriend,boys,brain,break,breaks,brian,bride,brilliant,bring,brings,british,broadway,broken,brother,brothers,brought,brutal,buddy,build,building,buried,business,businessman,buy,ca,california,call,called,calls,came,camp,captain,capture,captured,car,care,career,case,cash,cast,castle,cat,catch,caught,cause,causes,center,century,chain,challenge,champion,chance,change,changed,changes,chaos,character,characters,charge,charles,charlie,charming,chase,chicago,chief,child,childhood,children,china,chinese,choice,choose,chris,christmas,church,cia,city,civil,civil war,claim,claims,class,classic,clear,close,closer,club,co,coach,coast,cold,college,come,comedy,comes,comic,coming,commit,committed,common,community,company,competition,complete,complicated,computer,con,concert,conflict,confront,connection,consequences,conspiracy,contact,continues,control,convince,convinced,cop,cops,corporate,corrupt,could,count,country,couple,courage,course,court,cousin,cover,crash,crazy,create,created,creature,creatures,crew,crime,crimes,criminal,criminals,crisis,cross,cult,culture,curse,cut,dad,dance,dancer,danger,dangerous,daniel,danny,daring,dark,darkness,date,daughter,daughters,david,day,days,de,dead,deadly,deal,dealer,dealing,death,deaths,decide,decides,deep,defeat,demon,demons,desert,desire,desperate,desperately,despite,destiny,destroy,destroyed,destruction,detective,determined,develops,devil,diamond,die,died,dies,different,difficult,directed,director,disaster,discover,discovered,discovers,discovery,disease,divorce,doctor,documentary,dog,done,door,double,dr,drama,drawn,dream,dreams,drive,driven,driver,drug,drugs,due,dying,early,earth,east,easy,eccentric,eddie,edge,effort,eight,either,elite,elizabeth,else,embark,embarks,empire,encounter,encounters,end,ends,enemies,enemy,england,english,enough,enter,enters,entire,epic,era,escape,escaped,escapes,especially,estate,estranged,europe,eve,even,event,events,eventually,ever,every,everyone,everything,evidence,evil,ex,except,executive,existence,expedition,experience,experiences,experiment,expert,explores,extraordinary,eye,eyes,face,faces,fact,failed,faith,fall,fall love,falling,falls,falls love,fame,families,family,famous,fantasy,far,farm,fashion,fast,fate,father,fbi,fear,fears,feature,features,featuring,feel,feelings,feels,fellow,female,field,fight,fighter,fighting,fights,figure,filled,film,filmed,filmmaker,films,final,finally,financial,find,find way,finding,finds,fire,first,first time,five,flesh,flight,fly,follow,following,follows,food,footage,football,force,forced,forces,forest,forever,forget,form,former,fortune,found,four,france,francisco,frank,free,freedom,french,friend,friends,friendship,front,full,fun,funny,future,game,games,gang,gangster,gave,gay,general,generation,genius,george,german,germany,get,gets,getting,ghost,giant,gift,girl,girlfriend,girls,give,given,gives,giving,global,go,god,goes,going,gold,gone,good,got,government,great,greatest,green

In [28]:
y  = df_subset.iloc[:, 1500:]
y.head()

,action,adventure,animation,comedy,crime,documentary,drama,family,fantasy,foreign,history,horror,music,mystery,romance,science fiction,thriller,tv movie,war,western
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Splits and further

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=45)

In [30]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(16111, 1500)
(16111, 20)
(4028, 1500)
(4028, 20)


In [31]:
bs = DecisionTreeClassifier(random_state=11)
mtc = MultiOutputClassifier(bs)
mtc.fit(X_train, y_train)
y_pred = mtc.predict(X_test)

In [38]:
pickle.dump(mtc, open('flask_app/model.pkl', 'wb'))
pickle.dump(vect, open('flask_app/vect.pkl', 'wb'))
pickle.dump(ohe, open('flask_app/ohe.pkl', 'wb'))

In [33]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.13083416087388283
